In [26]:
import pandas as pd
import numpy as np
import os.path
import requests
import json

root_path = os.path.dirname(os.getcwd())

# Import food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

In [27]:
# Create weather dataframe with entry for each inspection
weather = pd.DataFrame({
    "inspection_id": inspections.inspection_id,
    "date": inspections.inspection_date,
})

features = [
    "precip_intensity",
    "temperature_max",
    "wind_speed",
    "humidity"
]

weather = pd.concat([weather, pd.DataFrame(columns=features)])

In [28]:
# Check if weather data exists, import or start empty dataframe
if os.path.isfile(os.path.join(root_path, "DATA/weather.csv")):
    old_weather = pd.read_csv(os.path.join(root_path, "DATA/weather.csv"))
else:
    old_weather = pd.DataFrame(columns=weather.columns)

In [29]:
KEY = "3454c4667481d2c0fec48f642aba2f18"

In [45]:
# Check if weather record can be found in old data.
# If not, request record from darksky.net
def get_weather(group):
    date = group.iloc[0].date
    record = old_weather[old_weather.date == date]
    if (not record.empty) & (not record.isnull().values.any()):
        print("record found")
        group.loc[:, features] = record.iloc[0][features].values
    else:
        url = "https://api.darksky.net/forecast/%s/41.836944,-87.684722,%s" % (KEY, date)
        response = requests.get(url).json()
        if "daily" in response:  
            print("response recieved")
            data = response.json()["daily"]["data"][0]
        
        group[features] = [
            data["precipIntensity"],
            data["temperatureMax"],
            data["windSpeed"],
            data["humidity"]
        ]
    return group

In [46]:
thing = weather.head(20).groupby(by="date").apply(get_weather)

request recieved
False
request recieved
False
record found
record found


In [38]:
thingle = response.json()

NameError: name 'response' is not defined

In [106]:
weather_data["daily"]["data"][0]

{'apparentTemperatureHigh': 69.98,
 'apparentTemperatureHighTime': 1529618400,
 'apparentTemperatureLow': 61.17,
 'apparentTemperatureLowTime': 1529672400,
 'apparentTemperatureMax': 69.98,
 'apparentTemperatureMaxTime': 1529618400,
 'apparentTemperatureMin': 65.02,
 'apparentTemperatureMinTime': 1529568000,
 'cloudCover': 0.97,
 'dewPoint': 64.07,
 'humidity': 0.93,
 'icon': 'rain',
 'moonPhase': 0.29,
 'ozone': 300.2,
 'precipIntensity': 0.0928,
 'precipIntensityMax': 0.2541,
 'precipIntensityMaxTime': 1529589600,
 'precipProbability': 0.99,
 'precipType': 'rain',
 'pressure': 1007.79,
 'summary': 'Heavy rain until evening, starting again overnight.',
 'sunriseTime': 1529576235,
 'sunsetTime': 1529631033,
 'temperatureHigh': 69.45,
 'temperatureHighTime': 1529618400,
 'temperatureLow': 60.88,
 'temperatureLowTime': 1529672400,
 'temperatureMax': 69.45,
 'temperatureMaxTime': 1529618400,
 'temperatureMin': 64.2,
 'temperatureMinTime': 1529568000,
 'time': 1529557200,
 'uvIndex': 6,
 '

In [118]:
# Save result
thing.to_csv(os.path.join(root_path, "DATA/weather.csv"), index=False)